In [ ]:
import pandas as pd
import scipy.stats as stats

In [ ]:
order_data = pd.read_csv('data/orders.csv')
browser_data = pd.read_csv('data/browsing.csv')

### Our Plan of Action

- Remove personal identifiers with the exception of the user ID, which is hashed so that if marketing team comes back with specific questions, we can try to help answer them
- If the user session came in with a tagged campaign, retain campaign information; otherwise leave blank
- Retain billing city and state, as this was specifically asked for by the marketing department to effectively answer questions for their campaigns
- Join order data and aggregate order quantity (number of orders) and order value (sum of quantities) per user
- Bound outliers that have exceptionally large or small amounts of customer value (if needed, you can create a written summary of these for marketing review)

In [ ]:
order_data.head()

In [ ]:
browser_data.head()

### First we drop the unnecessary PII columns

In [ ]:
browser_data = browser_data.drop(['browser_agent', 'ip'], axis=1)

In [ ]:
order_data = order_data.drop(['email', 'street_address'], axis=1)

In [ ]:
browser_and_orders = browser_data.merge(order_data, how='inner', on=['order_number'], 
                                        suffixes=('__browser','__order'))

In [ ]:
browser_and_orders.head()

In [ ]:
summary_by_order = browser_and_orders.groupby('order_number').agg({'user_id':'first', 
                                                'city':'first', 
                                                'state':'first', 
                                                'num_items':'sum',
                                                'total_price': 'sum',
                                                'timestamp__order': 'first',
                                                'inbound_uri': 'first'})

In [ ]:
summary_by_order.head()

### Encrypting the User ID

In [ ]:
summary_by_order.user_id.map(lambda x: len(x)).max()

In [ ]:
import string, random

def genkey(length: int):
    """Generate key."""
    return "".join(random.choice(string.ascii_lowercase + 
                                 string.ascii_uppercase + 
                                 string.digits + 
                                 '[@_!#$%^&*()<>?/\|}{~:]') for _ in range(length))

def xor_strings(s, t):
    """xor two strings together."""
    if isinstance(s, str):
        # Text strings contain single characters
        return "".join(chr(ord(a) ^ ord(b)) for a, b in zip(s, t))
    else:
        # Bytes objects contain integer values in the range 0-255
        return bytes([a ^ b for a, b in zip(s, t)])

In [ ]:
my_secret_key = genkey(20)

In [ ]:
def xor_username(key, string):
    if len(string) < 20:
        string += ' ' * (20 - len(string))
    return xor_strings(string, key)

In [ ]:
summary_by_order['e_user_id'] = summary_by_order.user_id.map(lambda x: xor_username(my_secret_key, x))

In [ ]:
summary_by_order = summary_by_order.drop(['user_id'], axis=1)

In [ ]:
xor_username(my_secret_key, summary_by_order['e_user_id'][1])

### Key Safety

What happens if we change or lose the key? :/ 

In [ ]:
backup = my_secret_key

In [ ]:
my_secret_key = genkey(20)

In [ ]:
xor_username(my_secret_key, summary_by_order['e_user_id'][1])

### Finding Bounds and Outliers

In [ ]:
summary_by_order.total_price.hist(bins=80)

In [ ]:
summary_by_order['z-score'] = stats.zscore(list(summary_by_order.total_price))

summary_by_order['z-score'].hist(bins=100)

In [ ]:
summary_by_order[summary_by_order.total_price > 27000].count()

In [ ]:
summary_by_order[summary_by_order.total_price < 1500].count()

In [ ]:
def remove_order_outliers(total):
    if total > 27000 or total < 1500:
        # NOTE: we want to watch this over time if we rerun the report and make sure it is fitting.
        # We could also normalize the data and use Interquartile Range (IQR) or Z-Scores 
        return True
    return False

In [ ]:
summary_by_order['remove'] = summary_by_order.total_price.map(remove_order_outliers)

In [ ]:
summary_by_order.count()

In [ ]:
summary_by_order = summary_by_order[summary_by_order['remove'] == False]

In [ ]:
summary_by_order = summary_by_order.drop(['remove'], axis=1)

### Getting started with Great Expectations

In [ ]:
import great_expectations as ge
context = ge.get_context()

In [ ]:
ge_df = ge.from_pandas(summary_by_order)

In [ ]:
ge_df.expect_column_values_to_be_between('total_price', 1500, 27000)

In [ ]:
ge_df.get_expectation_suite(discard_failed_expectations=False)


In [ ]:
import json
with open("order_summary_for_sharing_expecation_file.json", "w") as my_file:    
    my_file.write(        
        json.dumps(ge_df.get_expectation_suite().to_json_dict())
    )

In [ ]:
summary_by_order.to_csv('data/order_summary_for_sharing.csv')